In [25]:
import pandas as pd
import numpy as np
from glob import glob
import os

# this script is written in commit
# commit 27e9b2b1c36e319fd992240d78e9e946dc8d96a5
# revert to that to check/re-run/modify/etc 

In [26]:
combined_df = pd.read_csv('~/Desktop/tmp/combined_df_20201204.csv')

### bioSample

In [50]:
# split genotype into genotype1, perturb1, genotype2, perturb2
# split treatment
# correct mis-spellings in experimentDesign

# get list of biosample sheets
subdir = "bioSample"
biosample_sheet_list = glob("/home/chase/code/brentlab/database_files/bioSample/*")

treatment_split_dict = {"medium_categories": ["DMEM","PBS","RPMI","YPD"],
                        "temp_categories": ["37C","30C"],
                        "atmosphere_categories": ["CO2"],
                        "treatment_categories": ['EtOH', 'Estradiol', "estradiol", 'mockEstradiol', "cAMP", "CHX"], # if cAMP, take everything after also
                        "other_conditions": ["pH7"]
                       } # end treatment_split_dict

for file in biosample_sheet_list:
    biosample_sheet_df = pd.read_excel(file)
    new_col_dict = {"genotype1":         [None] * len(biosample_sheet_df.index) ,
                    "perturbation1":     [None] * len(biosample_sheet_df.index),
                    "genotype2":         [None] * len(biosample_sheet_df.index),
                    "perturbation2":     [None] * len(biosample_sheet_df.index),
                    "experiment_design": [None] * len(biosample_sheet_df.index),
                    "medium":            [None] * len(biosample_sheet_df.index),
                    "temperature":       [None] * len(biosample_sheet_df.index),
                    "atmosphere":        [None] * len(biosample_sheet_df.index),
                    "other_conditions":  [None] * len(biosample_sheet_df.index),
                    "treatment":         [None] * len(biosample_sheet_df.index) 
                   } # end new_col_dict
    
    # split genotype
    for index, row in biosample_sheet_df.iterrows():
        # split the genotype column
        geno_split = row.genotype.split(".")
        geno1 = "_".join(geno_split[0].split("_", 2)[:2])
        try:
            # if over is in first genotype, perturbation is over
            perturbation1 = geno_split[0].split("_")[2]
        except(IndexError):
            # otherwise, deletion
            if geno1 != "CNAG_00000" and geno1.startswith("CNAG"):
                perturbation1 = "deletion"
            else:
                perturbation1 = None
        try:
            geno2 = geno_split[1]
        except(IndexError):
            geno2 = None
        new_col_dict["genotype1"][index] = geno1
        new_col_dict["perturbation1"][index] = perturbation1
        new_col_dict["genotype2"][index] = geno2
        # true if there are any entries in genotype2 other than none. If there are, add "deletion"
        if any(new_col_dict["genotype2"]):
            new_col_dict["perturbation2"][index] = "deletion"
        
        # fix formatting in experimentDesign column
        if row.experimentDesign == "90minuteinduction":
            new_col_dict["experiment_design"][index] = "90minuteInduction"
        elif row.experimentDesign == "24hourinduction":
            new_col_dict["experiment_design"][index] = "24hourInduction"
        else:
            new_col_dict["experiment_design"][index] = row.experimentDesign
        
        # split treatment column
        treatment_split = row.treatment.split(".")
        if treatment_split[0] in treatment_split_dict["medium_categories"]:
            new_col_dict["medium"][index] = treatment_split[0]
        if treatment_split[0] in treatment_split_dict["temp_categories"]:
            new_col_dict["temperature"][index] = treatment_split[0].replace("C","")
        if treatment_split[0] in treatment_split_dict["treatment_categories"]:
            new_col_dict["treatment"][index] = treatment_split[0]
        # if index error, stop checking
        try:
            if treatment_split[1] in treatment_split_dict["temp_categories"]:
                new_col_dict["temperature"][index] = treatment_split[1].replace("C","")
            if treatment_split[1] in treatment_split_dict["atmosphere_categories"]:
                new_col_dict["atmosphere"][index] = treatment_split[1]
            if treatment_split[2] in treatment_split_dict["atmosphere_categories"]:
                new_col_dict["atmosphere"][index] = treatment_split[2]
            if treatment_split[2] in treatment_split_dict["treatment_categories"]:
                if treatment_split[2] == "cAMP":
                    new_col_dict["treatment"][index] = ".".join(treatment_split[2:len(treatment_split)])
                else:
                    new_col_dict["treatment"][index] = treatment_split[2]
            if treatment_split[2] in treatment_split_dict["other_conditions"]:
                new_col_dict["other_conditions"][index] = treatment_split[2]
            if treatment_split[3] in treatment_split_dict["other_conditions"]:
                new_col_dict["other_conditions"][index] = treatment_split[3]
            if treatment_split[3] in treatment_split_dict["treatment_categories"]:
                if treatment_split[3] == "cAMP":
                    new_col_dict["treatment"][index] = ".".join(treatment_split[3:len(treatment_split)])
                else:
                    new_col_dict["treatment"][index] = treatment_split[3]
            
        except(IndexError):
            pass

    biosample_sheet_df.experimentDesign = new_col_dict["experiment_design"]
    # list of new genotype columns
    new_genotype_columns = ["genotype1", "perturbation1", "genotype2", "perturbation2"]
    # add new genotype columns after if not entirely null at genotype location (note reverse order of new_genotype_columns)
    for new_col in new_genotype_columns:
        # returns true if any entries in col are other than None
        if any(new_col_dict[new_col]):
            biosample_sheet_df.insert(list(biosample_sheet_df.columns).index("genotype"), new_col, new_col_dict[new_col])
    # drop (old) genotype column
    # list of new treatment columns
    new_treatment_columns = ["medium", "temperature", "atmosphere", "treatment", "other_conditions"]
    # rename treatment to treatment_old
    biosample_sheet_df.rename(columns = {"treatment": "treatment_old"}, inplace = True) 
    # add new treatment columns if they are not entirely null after treatment_old (note reverse order of new_genotype_columns)
    for new_col in new_treatment_columns:
        # returns true if any entries in col are other than None
        if any(new_col_dict[new_col]):
            biosample_sheet_df.insert(list(biosample_sheet_df.columns).index("treatment_old"), new_col, new_col_dict[new_col])
    # write dataframe back to the same path
    biosample_sheet_df.to_excel(file, index=False)
            
        

In [10]:
combined_df.treatment.unique()

array(['DMEM.37C.CO2', 'PBS', 'DMEM.30C.CO2.pH7', 'DMEM.37C.CO2.pH7',
       'RPMI.30C.CO2.pH7', 'RPMI.37C.CO2.pH7', 'YPD.30C.CO2.pH7',
       'YPD.37C.CO2.pH7', 'DMEM.30C.CO2.cAMP', 'DMEM.37C.CO2.cAMP',
       'RPMI.30C.CO2.cAMP', 'RPMI.37C.CO2.cAMP', 'YPD.30C.CO2.cAMP',
       'YPD.37C.CO2.cAMP', '37C.CO2', 'EtOH', 'Estradiol',
       'mockEstradiol', 'DMEM.30C.cAMP', 'DMEM.37C.cAMP', 'RPMI.30C.cAMP',
       'RPMI.37C.cAMP', 'YPD.30C.cAMP', 'YPD.37C.cAMP', 'DMEM.30C',
       'DMEM.37C', 'RPMI.30C', 'RPMI.37C', 'YPD.30C', 'YPD.37C',
       'RPMI.30C.cAMP.0mM', 'RPMI.30C.cAMP.1.8mM', 'RPMI.30C.cAMP.3.3mM',
       'RPMI.30C.cAMP.6mM', 'RPMI.30C.cAMP.11mM', 'RPMI.30C.cAMP.20mM',
       '30C', 'estradiol', 'DMEM.30C.CO2', 'RPMI.30C.CO2', 'RPMI.37C.CO2',
       'YPD.30C.CO2', 'YPD.37C.CO2', '30C.CHX', '37C.CO2.CHX',
       'DMEM.30C.pH7', 'DMEM.37C.pH7', 'RPMI.30C.pH7', 'RPMI.37C.pH7',
       'YPD.30C.pH7', 'YPD.37C.pH7'], dtype=object)

In [ ]:
res_list = glob("/home/chase/Desktop/tmp/bioSample_test/*")

